In [65]:
# setup the const variables

# the path of the data
DATA_PATH = (
    "dataset/Image_Classification/dataset_24581896"
)


STEP_PER_EPOCH = 50
BATCH_SIZE = 25
EPOCH = 240

In [66]:
import tensorflow as tf
import tensorflow.keras.preprocessing as prep


def prepare_data(data_path, batch_size):
    # # create a image generator
    img_gen = prep.image.ImageDataGenerator(
        featurewise_center=True,
        # featurewise_std_normalization=True,
        rescale=1.0 / 255,
        rotation_range=30,
        width_shift_range=0.05,
        height_shift_range=0.05,    
        zoom_range=0.5  ,
        horizontal_flip=True,
        vertical_flip=True,
        validation_split=0.2,
    )

    # load the data
    train_ds = img_gen.flow_from_directory(
        data_path,
        class_mode="categorical",
        target_size=(256, 256),
        seed=24581896,
        subset="training",
        batch_size=batch_size,
    )

    val_ds = img_gen.flow_from_directory(
        data_path,
        class_mode="categorical",
        target_size=(256, 256),
        seed=24581896,
        subset="validation",
        batch_size=batch_size,
    )

    labels = train_ds.class_indices.keys()
    num_classes = len(labels)

    return train_ds, val_ds, labels, num_classes

In [67]:
train_ds, val_ds, class_names, num_classes = prepare_data(DATA_PATH, BATCH_SIZE)

Found 2596 images belonging to 20 classes.
Found 640 images belonging to 20 classes.


In [68]:
type(train_ds)
# train_ds.labels
train_ds.class_indices.keys()

dict_keys(['AMETHYST WOODSTAR', 'BLACK THROATED HUET', 'Bedlington_terrier', 'CAMPO FLICKER', 'DOWNY WOODPECKER', 'ELEGANT TROGON', 'EntleBucher', 'FIERY MINIVET', 'GREATER PRAIRIE CHICKEN', 'Great_Dane', 'KING EIDER', 'KING VULTURE', 'ORNATE HAWK EAGLE', 'Saint_Bernard', 'Shetland_sheepdog', 'Walker_hound', 'black-and-tan_coonhound', 'boxer', 'dingo', 'toy_terrier'])

## step 2. Create a Classifiers mdoel 

1. Customize AlexNet/GoogleNet/ResNet etc. and reduce/increase the layers, Train, and Test for image classification.

In [71]:
import tensorflow
import tensorflow.keras
from tensorflow.keras import models, layers, applications as apps, callbacks
import os

from datetime import datetime

# from models.inception_v4 import inception_v4

# resnet implement


def view_train_chart(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(epochs, loss, "r", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()

def create_model__resnetv2(num_classes):
    # init
    model = models.Sequential(
        [
            layers.Input((256, 256, 3)),
            apps.ResNet152V2(
                include_top=False,
                weights="imagenet",
                pooling="max",
            ),
            layers.Flatten(),
            layers.Dense(4096, activation="relu"),
            layers.Dense(num_classes, activation="softmax", name="predictions"),
        ]
    )

    return model


# google net (inception_v4)
def create_model__inception_v3(num_classes):
    model = models.Sequential(
        [
            layers.Input((256, 256, 3)),
            apps.InceptionV3(
                include_top=False,
                weights="imageNet",
            ),
            layers.Flatten(),
            layers.Dense(4096, activation="relu"),
            layers.Dense(num_classes, activation="softmax", name="predictions"),
        ]
    )
    # include_top = (True,)
    # weights = ("imagenet",)
    # input_tensor = (None,)
    # input_shape = (None,)
    # pooling = (None,)
    # classes = (1000,)
    # classifier_activation = "softmax"
    return model


def train_model(model,model_name, train_ds, val_ds):
    # init

    now_time = f"{datetime.now():%Y-%m-%d_%H-%M-%S}"

    log_file_name = f"./logs/training__{model_name}__{now_time}.csv"

    open(log_file_name, 'a').close()

    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["acc"],
    )

    ckp_weights_dir = f"./ckp_weights/img_clsr/{model_name}__{now_time}/"
    os.makedirs(ckp_weights_dir, exist_ok=True)

    checkpoint = callbacks.ModelCheckpoint(
        ckp_weights_dir + "{epoch:02d}-{val_loss:.2f}.keras",
        monitor="val_loss",
        verbose=0,
        save_best_only=False,
        save_weights_only=False,
        mode="auto",
        save_freq="epoch",
    )

    cv_logging = callbacks.CSVLogger(log_file_name, separator=",", append=True)
    history = model.fit(
        train_ds,
        steps_per_epoch=STEP_PER_EPOCH,  # 2000 images = batch_size * steps
        epochs=EPOCH,
        validation_data=val_ds,
        validation_steps=STEP_PER_EPOCH//5,  # 1000 images = batch_size * steps
        callbacks=[checkpoint, cv_logging],
    )

    return model, history

In [72]:
# test run, with cpu

# train_ds, val_ds, class_names, num_classes
# with tensorflow.device("/CPU:0"):
mod_name = "ResNet151V2"
resnet_mod = create_model__resnetv2(num_classes)
resnet_mod, hist = train_model(
    resnet_mod, model_name=mod_name, train_ds=train_ds, val_ds=val_ds
)
view_train_chart(hist)
resnet_mod.save(f"./models/img_clsr/{mod_name}.keras")

Epoch 1/120


/home/ghost/anaconda3/envs/uts-dl/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 387s 7s/step - acc: 0.0708 - loss: 12.5704 - val_acc: 0.0600 - val_loss: 65739173888.0000
Epoch 2/120
20/50 ━━━━━━━━━━━━━━━━━━━━ 3:17 7s/step - acc: 0.0893 - loss: 3.1617

KeyboardInterrupt: 

In [ ]:
# # Loads the weights
# resnet_mod = create_model__resnetv2(num_classes)

# resnet_mod.load_weights(
#     "./weights/img_clsr__ResNet151V2__2024-04-29_04-15-20_.120-2.99.keras"
# )
# # Evaluate the model
# loss, acc = resnet_mod.evaluate(, verbose=2)
# print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

In [3]:
# train_ds, val_ds, class_names, num_classes
# with tensorflow.device("/CPU:0"):
mod_name = "InceptionV3"
inception_mod = create_model__inception_v3(num_classes)
inception_mod, hist = train_model(
    inception_mod, model_name=mod_name, train_ds=train_ds, val_ds=val_ds
)
view_train_chart(hist)
inception_mod.save(f"./models/img_clsr/{mod_name}.keras")